In [6]:
import os
import openai
import base64
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv

# Cargar .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Inicializar cliente OpenAI
client = openai.OpenAI(api_key=api_key)

# 📸 Ruta de imagen
image_path = "./encabezado.jpg"

# 🔧 Redimensionar en memoria
def resize_image_in_memory(image_path, resize_percent=50):
    img = Image.open(image_path)
    original_size = img.size
    
    new_width = int(original_size[0] * resize_percent / 100)
    new_height = int(original_size[1] * resize_percent / 100)
    img = img.resize((new_width, new_height), Image.LANCZOS)
    
    # Convertir a bytes en memoria
    buffer = BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    
    print(f"📏 Imagen original: {original_size}, redimensionada a: ({new_width}, {new_height})")
    return buffer

# 🧪 Codificar imagen desde buffer
def encode_image_base64_from_buffer(buffer):
    return base64.b64encode(buffer.read()).decode("utf-8")

# 🧠 Extraer texto con modelo seleccionable
def extract_text_from_image(base64_image, model="gpt-4o", max_tokens=2000):
    # Modelos disponibles con visión y sus precios por 1K tokens
    vision_models = {
        "gpt-5": {
            "input": 0.00125, "output": 0.010,
            "use_max_completion_tokens": True
        },
        "gpt-5-mini": {
            "input": 0.00025, "output": 0.002,
            "use_max_completion_tokens": True
        },
        "gpt-4o": {
            "input": 0.005, "output": 0.015,
            "use_max_completion_tokens": False
        },
        "gpt-4o-mini": {
            "input": 0.00015, "output": 0.0006,
            "use_max_completion_tokens": False
        },
        "gpt-4-turbo": {
            "input": 0.01, "output": 0.03,
            "use_max_completion_tokens": False
        },
        "gpt-4-vision-preview": {
            "input": 0.01, "output": 0.03,
            "use_max_completion_tokens": False
        }
    }

    # Validación modelo
    if model not in vision_models:
        print(f"⚠️ Modelo {model} no soporta visión. Usando gpt-4o")
        model = "gpt-4o"

    pricing = vision_models[model]
    print(f"🤖 Usando modelo: {model}")

    # Construcción de request compatible con todos los modelos
    request_params = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "En base a la imagen extrae un json con la fecha, hora, presidente y asunto. Solo quiero el json sin comentarios adicionales."},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ]
            }
        ]
    }

    # ⚠️ Diferencia clave: GPT-5 usa max_completion_tokens
    if pricing["use_max_completion_tokens"]:
        request_params["max_completion_tokens"] = max_tokens
    else:
        request_params["max_tokens"] = max_tokens

    # Llamada a la API
    response = client.chat.completions.create(**request_params)

    # Cálculo de costos
    usage = response.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = prompt_tokens + completion_tokens

    cost = (prompt_tokens * pricing["input"] + completion_tokens * pricing["output"]) / 1000
    print(f"\n📊 Tokens usados: prompt={prompt_tokens}, completion={completion_tokens}, total={total_tokens}")
    print(f"💵 Costo real estimado: ${cost:.5f} USD")

    return response.choices[0].message.content

# 🚀 Función principal para probar diferentes configuraciones
def process_image_ocr(image_path, resize_percent=40, model="gpt-4o", max_tokens=2000):
    """
    Procesa una imagen con OCR usando diferentes modelos y configuraciones
    
    Args:
        image_path: Ruta a la imagen
        resize_percent: Porcentaje de redimensionado (100 = tamaño original)
        model: Modelo de OpenAI a usar ('gpt-4o', 'gpt-4o-mini', etc.)
        max_tokens: Máximo de tokens en la respuesta
    """
    # Redimensionar en memoria
    img_buffer = resize_image_in_memory(image_path, resize_percent)
    
    # Codificar a base64
    b64_img = encode_image_base64_from_buffer(img_buffer)
    
    # Extraer texto con el modelo seleccionado
    result = extract_text_from_image(b64_img, model, max_tokens)
    
    return result

# ▶️ Ejemplos de uso
if __name__ == "__main__":
    # Probar con diferentes modelos
    modelos_a_probar = ["gpt-4o", "gpt-4o-mini","gpt-5","gpt-5-mini"]
    
    for modelo in modelos_a_probar:
        print(f"\n{'='*50}")
        print(f"🔬 Probando con modelo: {modelo}")
        print(f"{'='*50}")
        
        ocr_result = process_image_ocr(
            image_path=image_path,
            resize_percent=50,
            model=modelo,
            max_tokens=2000
        )
        
        print(f"\n📤 Resultado OCR con {modelo}:\n")
        print(ocr_result[:500] + "..." if len(ocr_result) > 500 else ocr_result)
        print("\n")


🔬 Probando con modelo: gpt-4o
📏 Imagen original: (2456, 537), redimensionada a: (1228, 268)
🤖 Usando modelo: gpt-4o

📊 Tokens usados: prompt=629, completion=155, total=784
💵 Costo real estimado: $0.00547 USD

📤 Resultado OCR con gpt-4o:

```json
{
  "fecha": "10/05/2024",
  "hora": "10:29 am",
  "presidente": "SOTO REYES, ALEJANDRO",
  "asunto": "TEXTO SUSTITUTORIO DEL PROYECTO DE LEY 7716, QUE PROPONE MODIFICAR LOS ARTÍCULOS 9 Y 72, LITERAL F), E INCORPORAR LA DÉCIMO TERCERA DISPOSICIÓN COMPLEMENTARIA TRANSITORIA EN LA LEY 30916, LEY ORGÁNICA DE LA JUNTA NACIONAL DE JUSTICIA, PARA MEJORAR SU FUNCIONAMIENTO Y SOSTENIBILIDAD"
}
```



🔬 Probando con modelo: gpt-4o-mini
📏 Imagen original: (2456, 537), redimensionada a: (1228, 268)
🤖 Usando modelo: gpt-4o-mini

📊 Tokens usados: prompt=19868, completion=155, total=20023
💵 Costo real estimado: $0.00307 USD

📤 Resultado OCR con gpt-4o-mini:

```json
{
  "fecha": "10/05/2024",
  "hora": "10:29 am",
  "presidente": "SOTO REYES, ALEJANDRO",
  